In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.9.1
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.23.0
pandas 1.4.3
sklearn 1.1.1
tensorflow 2.9.1
keras.api._v2.keras 2.9.0


In [2]:
!ls generate_csv

test_00.csv
test_01.csv
test_02.csv
test_03.csv
test_04.csv
test_05.csv
test_06.csv
test_07.csv
test_08.csv
test_09.csv
train_00.csv
train_01.csv
train_02.csv
train_03.csv
train_04.csv
train_05.csv
train_06.csv
train_07.csv
train_08.csv
train_09.csv
train_10.csv
train_11.csv
train_12.csv
train_13.csv
train_14.csv
train_15.csv
train_16.csv
train_17.csv
train_18.csv
train_19.csv
valid_00.csv
valid_01.csv
valid_02.csv
valid_03.csv
valid_04.csv
valid_05.csv
valid_06.csv
valid_07.csv
valid_08.csv
valid_09.csv


In [3]:
source_dir = "./generate_csv/"

# 通过判断开头去添加文件
def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results

train_filenames = get_filenames_by_prefix(source_dir, "train")
valid_filenames = get_filenames_by_prefix(source_dir, "valid")
test_filenames = get_filenames_by_prefix(source_dir, "test")

import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)

['./generate_csv/train_00.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_04.csv',
 './generate_csv/train_05.csv',
 './generate_csv/train_06.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_08.csv',
 './generate_csv/train_09.csv',
 './generate_csv/train_10.csv',
 './generate_csv/train_11.csv',
 './generate_csv/train_12.csv',
 './generate_csv/train_13.csv',
 './generate_csv/train_14.csv',
 './generate_csv/train_15.csv',
 './generate_csv/train_16.csv',
 './generate_csv/train_17.csv',
 './generate_csv/train_18.csv',
 './generate_csv/train_19.csv']
['./generate_csv/valid_00.csv',
 './generate_csv/valid_01.csv',
 './generate_csv/valid_02.csv',
 './generate_csv/valid_03.csv',
 './generate_csv/valid_04.csv',
 './generate_csv/valid_05.csv',
 './generate_csv/valid_06.csv',
 './generate_csv/valid_07.csv',
 './generate_csv/valid_08.csv',
 './generate_csv/valid_09.csv']
['./generate_csv/test_00.csv',
 './gener

In [4]:
def parse_csv_line(line, n_fields = 9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)


In [5]:
!ls

generate_csv
generate_tfrecords
temp.csv
tf01-dataset_basic_api.ipynb
tf02_data_generate_csv.ipynb
tf03-tfrecord_basic_api.ipynb
tf04_data_generate_tfrecord.ipynb
tfrecord_basic


# 把train_set,valid_set,test_set 存储到tfrecord类型的文件中

In [6]:
# 把基础的如何序列化的步骤搞到一个函数
def serialize_example(x, y):
    """Converts x, y to tf.train.Example and serialize"""
    input_feautres = tf.train.FloatList(value = x)  # 特征
    label = tf.train.FloatList(value = y)  # 标签
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(
                float_list = input_feautres),
            "label": tf.train.Feature(float_list = label)
        }
    )
    # 把features变为example
    example = tf.train.Example(features = features)
    return example.SerializeToString()  # 把example序列化

# n_shards是存为多少个文件，steps_per_shard和 steps_per_epoch类似
def csv_dataset_to_tfrecords(base_filename, dataset,
                             n_shards, steps_per_shard,
                             compression_type = None):
    # 压缩文件类型
    options = tf.io.TFRecordOptions(
        compression_type = compression_type)
    all_filenames = []
    
    for shard_id in range(n_shards):
        filename_fullpath = '{}_{:05d}-of-{:05d}'.format(
            base_filename, shard_id, n_shards)  # base_filename是一个前缀
        # 打开文件
        with tf.io.TFRecordWriter(filename_fullpath, options) as writer:
            # 取出数据,为什么skip，上一个文件写了前500行，下一个文件存后面的数据
            for x_batch, y_batch in dataset.skip(shard_id * steps_per_shard).take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(
                        serialize_example(x_example, y_example))
        all_filenames.append(filename_fullpath)
    # 返回所有tfrecord文件名
    return all_filenames

In [7]:
!rm -rf generate_tfrecords

In [8]:
for i in train_set.take(1):
    print(i)  

(<tf.Tensor: shape=(32, 8), dtype=float32, numpy=
array([[-1.05917811e+00,  1.39356470e+00, -2.63319686e-02,
        -1.10067599e-01, -6.13819897e-01, -9.69593525e-02,
         3.24713111e-01, -3.74772437e-02],
       [-1.07750773e+00, -4.48740691e-01, -5.68056822e-01,
        -1.42692626e-01, -9.66667682e-02,  1.23264685e-01,
        -3.14486384e-01, -4.81895894e-01],
       [ 8.01544309e-01,  2.72161424e-01, -1.16243929e-01,
        -2.02311516e-01, -5.43051600e-01, -2.10396163e-02,
        -5.89762092e-01, -8.24184567e-02],
       [ 4.85305160e-01, -8.49241912e-01, -6.53012618e-02,
        -2.33796556e-02,  1.49743509e+00, -7.79065788e-02,
        -9.02363241e-01,  7.81451464e-01],
       [-1.11997497e+00, -1.32984328e+00,  1.41900450e-01,
         4.65813696e-01, -1.03017777e-01, -1.07441843e-01,
        -7.95052409e-01,  1.53047168e+00],
       [-2.22356573e-01,  1.39356470e+00,  2.99129952e-02,
         8.01452026e-02, -5.09481966e-01, -6.23859912e-02,
        -8.65037739e-01,  8

In [9]:
%%time
# 训练集和测试集都分20
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // 10
test_steps_per_shard = 5170 // batch_size // 10

output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, 10, valid_steps_per_shard, None)
test_tfrecord_fielnames = csv_dataset_to_tfrecords(
    test_basename, test_set, 10, test_steps_per_shard, None)
# 执行会发现目录下总计生成了60个文件,这里文件数目改为一致，为了对比时间

CPU times: total: 41.6 s
Wall time: 31.1 s


In [10]:
!ls -l generate_tfrecords

total 1960
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00000-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00001-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00002-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00003-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00004-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00005-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00006-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00007-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00008-of-00010
-rw-r--r-- 1 yf 197121 47616 Jul 28 00:02 test_00009-of-00010
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00000-of-00020
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00001-of-00020
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00002-of-00020
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00003-of-00020
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00004-of-00020
-rw-r--r-- 1 yf 197121 53568 Jul 28 00:01 train_00005-

In [11]:
# 生成一下压缩的
# n_shards = 20
# train_steps_per_shard = 11610 // batch_size // n_shards
# valid_steps_per_shard = 3880 // batch_size // n_shards
# test_steps_per_shard = 5170 // batch_size // n_shards

# output_dir = "generate_tfrecords_zip"
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)

# train_basename = os.path.join(output_dir, "train")
# valid_basename = os.path.join(output_dir, "valid")
# test_basename = os.path.join(output_dir, "test")
# 只需修改参数的类型即可
# train_tfrecord_filenames = csv_dataset_to_tfrecords(
#     train_basename, train_set, n_shards, train_steps_per_shard,
#     compression_type = "GZIP")
# valid_tfrecord_filenames = csv_dataset_to_tfrecords(
#     valid_basename, valid_set, n_shards, valid_steps_per_shard,
#     compression_type = "GZIP")
# test_tfrecord_fielnames = csv_dataset_to_tfrecords(
#     test_basename, test_set, n_shards, test_steps_per_shard,
#     compression_type = "GZIP")

In [12]:
!ls -l generate_tfrecords_zip

ls: cannot access 'generate_tfrecords_zip': No such file or directory


In [13]:
# 打印一下文件名
pprint.pprint(train_tfrecord_filenames)
pprint.pprint(valid_tfrecord_filenames)
pprint.pprint(test_tfrecord_fielnames)

['generate_tfrecords\\train_00000-of-00020',
 'generate_tfrecords\\train_00001-of-00020',
 'generate_tfrecords\\train_00002-of-00020',
 'generate_tfrecords\\train_00003-of-00020',
 'generate_tfrecords\\train_00004-of-00020',
 'generate_tfrecords\\train_00005-of-00020',
 'generate_tfrecords\\train_00006-of-00020',
 'generate_tfrecords\\train_00007-of-00020',
 'generate_tfrecords\\train_00008-of-00020',
 'generate_tfrecords\\train_00009-of-00020',
 'generate_tfrecords\\train_00010-of-00020',
 'generate_tfrecords\\train_00011-of-00020',
 'generate_tfrecords\\train_00012-of-00020',
 'generate_tfrecords\\train_00013-of-00020',
 'generate_tfrecords\\train_00014-of-00020',
 'generate_tfrecords\\train_00015-of-00020',
 'generate_tfrecords\\train_00016-of-00020',
 'generate_tfrecords\\train_00017-of-00020',
 'generate_tfrecords\\train_00018-of-00020',
 'generate_tfrecords\\train_00019-of-00020']
['generate_tfrecords\\valid_00000-of-00010',
 'generate_tfrecords\\valid_00001-of-00010',
 'generate

In [14]:
%%time
# 把数据读取出来
expected_features = {
    "input_features": tf.io.FixedLenFeature([8], dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"], example["label"]

def tfrecords_reader_dataset(filenames, n_readers=5,
                             batch_size=32, n_parse_threads=5,
                             shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()  # 为了能够无限次epoch
    dataset = dataset.interleave(
#         lambda filename: tf.data.TFRecordDataset(
#             filename, compression_type = "GZIP"),
          lambda filename: tf.data.TFRecordDataset(
            filename),
          cycle_length = n_readers
    )
    # 洗牌，就是给数据打乱,样本顺序打乱
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example,
                          num_parallel_calls=n_parse_threads)  # 把对应的一个样本是字节流的，变为浮点类型
    dataset = dataset.batch(batch_size)  # 原来写进去是一条一条的sample，要分配
    return dataset

# 测试一下，tfrecords_reader_dataset是否可以正常运行
# tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames,
#                                            batch_size = 3)
# for x_batch, y_batch in tfrecords_train.take(10):
#     print(x_batch)
#     print(y_batch)

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
%%time
# 得到dataset,dataset是tensor，可以直接拿tensor训练

batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(
    train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(
    valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(
    test_tfrecord_fielnames, batch_size = batch_size)

Cause: could not parse the source code of <function tfrecords_reader_dataset.<locals>.<lambda> at 0x00000246BE5BB940>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function tfrecords_reader_dataset.<locals>.<lambda> at 0x00000246BE5BB940>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function parse_example at 0x00000246BE5BB550>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.auto

In [16]:
type(tfrecords_train_set)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [17]:
for i in tfrecords_train_set.take(1):
    print(i)

(<tf.Tensor: shape=(32, 8), dtype=float32, numpy=
array([[-2.77109686e-02,  1.87416613e+00, -2.01294750e-01,
         2.25147326e-02,  1.41042203e-01, -1.04131013e-01,
         9.96572435e-01, -1.44064176e+00],
       [-3.91372681e-01,  3.18607129e-02, -3.26981932e-01,
        -5.25816232e-02, -5.13111115e-01,  3.54368538e-02,
         5.95323086e-01, -1.08111203e+00],
       [-1.10068619e+00,  1.23336422e+00, -4.57008183e-01,
        -2.96145111e-01, -8.38826895e-01,  8.82104039e-02,
        -7.20401347e-01,  1.13598776e+00],
       [-8.39276016e-01,  1.47366500e+00, -6.63657010e-01,
        -2.21059039e-01, -5.04945576e-01,  1.01379789e-01,
        -7.53061175e-01,  6.81582153e-01],
       [-6.91466868e-01,  6.72662616e-01, -3.54697227e-01,
        -1.24215007e-01,  4.97605681e-01,  1.00189252e-02,
        -7.29732752e-01,  7.36510277e-01],
       [ 9.33741331e-01, -4.48740691e-01,  1.85706854e-01,
        -6.31846488e-02, -1.69249669e-01,  1.85529351e-01,
        -6.36418939e-01,  5

In [18]:
 #开始训练
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

history = model.fit(tfrecords_train_set,
                    validation_data = tfrecords_valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Epoch 1/100
348/348 [==============================] - 1s 1ms/step - loss: 0.9218 - val_loss: 1.2317
Epoch 2/100
348/348 [==============================] - 0s 1ms/step - loss: 0.7446 - val_loss: 0.9708
Epoch 3/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4796 - val_loss: 0.5078
Epoch 4/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4417 - val_loss: 0.4749
Epoch 5/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4205 - val_loss: 0.4618
Epoch 6/100
348/348 [==============================] - 0s 1ms/step - loss: 0.4094 - val_loss: 0.4468
Epoch 7/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3966 - val_loss: 0.4394
Epoch 8/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3960 - val_loss: 0.4305
Epoch 9/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3924 - val_loss: 0.4229
Epoch 10/100
348/348 [==============================] - 0s 1ms/step - loss: 0.3852 - val_lo

In [19]:
model.evaluate(tfrecords_test_set, steps = 5160 // batch_size)

161/161 [==============================] - 0s 725us/step - loss: 0.3824


0.3824114501476288